# Assignment-1
## Team: Aditya Ahuja (2020275), Deeptanshu Barman Chowdhuri (2020293)

##  Imports & Dataset

In [1]:
!pip install torchmetrics
!pip install lightning

   ---------------------------------------- 0.0/365.3 kB ? eta -:--:--
   --------------------------------------- 365.3/365.3 kB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
   ---------------------------------------- 60.8/60.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/50.5 kB ? eta -:--:--
   ---------------------------------------- 50.5/50.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/76.7 kB ? eta -:--:--
   ---------------------------------------- 76.7/76.7 kB 4.4 MB/s eta 0:00:00


In [2]:
import torch
import requests
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import lightning as L

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [5]:
Noise_0_data = requests.get(
    "http://AdityaAhuja01.pythonanywhere.com/data/df_syn_train_0_0_.csv"
)
Noise_Low_data = requests.get(
    "http://AdityaAhuja01.pythonanywhere.com/data/df_synA_train_shuffled.csv"
)
Noise_High_data = requests.get(
    "http://AdityaAhuja01.pythonanywhere.com/data/df_synA_test_hard_shuffled_sample.csv"
)

if Noise_0_data.status_code == 200 and Noise_Low_data.status_code == 200 and Noise_High_data.status_code == 200:
    datafolder = "Data/Assignment1"

    if not os.path.exists(datafolder):
        os.makedirs(datafolder)

    with open(os.path.join(datafolder, "data_0_noise"), "wb") as f:
        f.write(Noise_0_data.text.encode("utf-8"))

    with open(os.path.join(datafolder, "data_Low_noise"), "wb") as f:
        f.write(Noise_Low_data.text.encode("utf-8"))

    with open(os.path.join(datafolder, "data_High_noise"), "wb") as f:
        f.write(Noise_High_data.text.encode("utf-8"))
else:
    print("Error in fetching data")

In [4]:
Noise_0_dataframe = pd.read_csv("Data/Assignment1/data_0_noise")
Noise_Low_dataframe = pd.read_csv("Data/Assignment1/data_Low_noise")
Noise_High_dataframe = pd.read_csv("Data/Assignment1/data_High_noise")


In [5]:
class_index = list(Noise_0_dataframe["era"].unique())
print(class_index)
def encode(value, class_index = class_index):
    return class_index.index(value)

Noise_0_dataframe["era"] = Noise_0_dataframe["era"].apply(encode)

[2, 4, 5, 7, 9, 10, 12, 15, 16, 18, 19, 21]


## Setting up Dataset & Dataloaders

In [7]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, noise, transform=None, target_transform=None,drop = None):
        self.dataframe = dataframe
        if drop != None:
            self.X = dataframe.drop(drop, axis=1).values
        else:
            self.X = dataframe.values
        self.y = dataframe["era"].values
        self.transform = transform
        self.target_transform = target_transform
        self.noise = noise

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item, label = self.X[idx], self.y[idx]
        return item, label

    def get_noise(self):
        return self.noise


In [8]:
#Setting up datasets
Noise_0_dataset = CustomDataset(Noise_0_dataframe, "0",drop = ["row_num","day","era","target_10_val","target_5_val"])
# Noise_Low_dataset = CustomDataset(Noise_Low_dataframe, "Low", drop = ["row_num","day","era","target_10_val","target_5_val"])
# Noise_High_dataset = CustomDataset(Noise_High_dataframe, "High", drop = ["row_num","day","era","target_10_val","target_5_val"])
Noise_0_train, Noise_0_test = random_split(Noise_0_dataset, [int(0.8 * len(Noise_0_dataset)), len(Noise_0_dataset) - int(0.8 * len(Noise_0_dataset))])
# Noise_Low_train, Noise_Low_test = random_split(Noise_Low_dataset, [int(0.8 * len(Noise_Low_dataset)), len(Noise_Low_dataset) - int(0.8 * len(Noise_Low_dataset))])
# Noise_High_train, Noise_High_test = random_split(Noise_High_dataset, [int(0.8 * len(Noise_High_dataset)), len(Noise_High_dataset) - int(0.8 * len(Noise_High_dataset))])

In [9]:
#Setting up dataloaders
Noise_0_train_loader = DataLoader(Noise_0_train, batch_size=64, shuffle=True)
Noise_0_test_loader = DataLoader(Noise_0_test, batch_size=64, shuffle=True)
# Noise_Low_train_loader = DataLoader(Noise_Low_train, batch_size=64, shuffle=True)
# Noise_Low_test_loader = DataLoader(Noise_Low_test, batch_size=64, shuffle=True)
# Noise_High_train_loader = DataLoader(Noise_High_train, batch_size=64, shuffle=True)
# Noise_High_test_loader = DataLoader(Noise_High_test, batch_size=64, shuffle=True)

In [11]:
Noise_0_dataframe.head()

,Open_n_val,High_n_val,Low_n_val,Close_n_val,Volume_n_val,SMA_10_val,SMA_20_val,CMO_14_val,High_n-Low_n_val,Open_n-Close_n_val,...,Open_n-Close_n_changelen_val,SMA_20-SMA_10_changelen_val,Close_n_slope_3_changelen_val,Close_n_slope_5_changelen_val,Close_n_slope_10_changelen_val,row_num,day,era,target_10_val,target_5_val
0,0.5,0.5,0.5,0.5,0.0,0.5,0.25,1.0,1.0,0.5,...,0.50,0.75,0.25,0.75,0.50,75,537,0,0.75,0.75
1,0.5,0.5,0.5,0.5,0.0,0.5,0.50,1.0,1.0,0.5,...,0.50,0.50,0.75,0.50,0.50,76,537,0,0.75,0.75
2,0.5,0.5,0.5,0.5,0.0,0.5,0.50,1.0,1.0,0.5,...,0.50,0.25,0.50,0.50,0.75,77,537,0,0.75,0.75
3,0.5,0.5,0.5,0.5,0.0,0.5,0.50,1.0,1.0,0.5,...,0.25,0.25,0.50,0.75,0.50,78,537,0,0.75,0.75
4,0.5,0.5,0.5,0.5,0.0,0.5,0.50,1.0,1.0,0.5,...,0.75,0.25,0.50,0.50,0.50,79,537,0,0.75,0.75


## Model

In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [14]:
class MyMLP(torch.nn.Module):
    def __init__ (self, modules):
        super().__init__()
        self.relu = torch.nn.ReLU()
        MyModuleList = torch.nn.ModuleList([m for m in modules])
        self.layers = torch.nn.Sequential(*MyModuleList)
        self.softmax = torch.nn.Softmax(dim = 1)


    def forward(self, X: torch.Tensor):
        if (X.shape[1] != 24):
            raise ValueError("Input shape must be (batch_size, 24)")
        X = X.to(device)
        X = self.layers(X)
        X = self.softmax(X)

        return X



In [15]:
Model = MyMLP([torch.nn.Linear(24,64),torch.nn.ReLU(),torch.nn.Linear(64,64),torch.nn.ReLU(),torch.nn.Linear(64,12)])
Model = Model.to(device)

## Lightning

In [16]:
from lightning.pytorch.utilities.types import OptimizerLRScheduler


class LightningMLP(L.LightningModule):
    def __init__(self, model, loss, optimizer):
        super().__init__()
        self.model = model
        self.loss = loss
        self.optimizer = optimizer

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        inputs, targets = torch.tensor(inputs).to(torch.float32),torch.tensor(targets).to(torch.long)
        inputs, targets = inputs.to(device),targets.to(device)
        output = self.model(inputs)
        loss = self.loss(output, targets)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return self.optimizer


In [17]:
mymodel = LightningMLP(Model, torch.nn.NLLLoss(), torch.optim.Adam(Model.parameters(), lr=0.001))
trainer = L.Trainer(max_epochs = 10)
trainer.fit(mymodel, Noise_0_train_loader, Noise_0_test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Anaconda\envs\amlenv\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
c:\Anaconda\envs\amlenv\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.se

Epoch 0:   0%|          | 0/98 [00:00<?, ?it/s] 

C:\Users\Aditya Ahuja\AppData\Local\Temp\ipykernel_11172\1612756171.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, targets = torch.tensor(inputs).to(torch.float32),torch.tensor(targets).to(torch.long)


Epoch 9: 100%|██████████| 98/98 [00:00<00:00, 176.41it/s, v_num=33]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 98/98 [00:00<00:00, 174.53it/s, v_num=33]
